In [1]:
import pandas as pd
import numpy as np

In [2]:
input = "need motgage for properety"

## Spell Corrector

In [3]:
from spellchecker import SpellChecker

In [4]:
spell = SpellChecker()


def correct_spelling(text, custom_words=None):
    # Add custom words to the dictionary
    if custom_words:
        for word in custom_words:
            spell.word_frequency.load_words([word])

    # Tokenize the text
    words = text.split()
    corrected_text = []
    for word in words:
        # Check if the word is misspelled
        corrected_word = spell.correction(word)
        corrected_text.append(corrected_word)
    # Join the corrected words back into a string
    corrected_text = ' '.join(corrected_text)
    return corrected_text

In [5]:
correct_spelling(input, custom_words=["EBM"])

'need mortgage for property'

## Stop Words Remover

In [13]:
from nltk.corpus import stopwords

In [18]:
stop_words = set(stopwords.words('english'))
len(stop_words)

179

In [20]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

len(ENGLISH_STOP_WORDS)

318

## Read Text Files

In [6]:
import glob

In [41]:
txt_files = glob.glob("..\\FAQs\\*.txt")

In [42]:
lines = set(sum([open(f, "r", encoding='utf-8').readlines() for f in glob.glob("..\\FAQs\\*.txt")],[]))
#lines.remove("\n")

In [43]:
ds = pd.DataFrame({"Questions": list(lines)})

In [44]:
ds["Questions"] = ds["Questions"].apply(lambda x: x.strip())

In [54]:
ds[ds["Questions"]=="How to get a mortgage"]

,Questions
162,How to get a mortgage


In [50]:
from sentence_transformers import SentenceTransformer, InputExample
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-mpnet-base-v2')

In [84]:
# Define some example data (replace with your actual dataset)
sentences = [
    "What is the bank's current interest rate for a savings account?",
    "How do I open a new checking account?",
    "What are the fees associated with international wire transfers?",
    "I forgot my debit card PIN. How can I reset it?",
    "What happens if my debit card is lost or stolen?",
    "How do I get a mortgage?"
]
labels = [1, 0, 0, 1, 0]  # 1 indicates similar meaning, 0 otherwise (replace with actual labels)

# Preprocess the text data (replace with your preprocessing steps)
# ... (e.g., remove stop words, lowercase, etc.)

# Encode the sentences using the pre-trained model
sentence_embeddings = model.encode(sentences)

In [51]:
sentences = ds["Questions"].tolist()

sentence_embeddings = model.encode(sentences)

In [64]:
# Function to calculate cosine similarity between embeddings
def calculate_similarity(query_embedding, sentence_embeddings):
  similarities = cosine_similarity(query_embedding.reshape(1, -1), sentence_embeddings)
  return similarities.flatten()

# Example usage: Find similar FAQs for a user query
user_query = "need mortgage"
user_query_embedding = model.encode(user_query)

similarities = calculate_similarity(user_query_embedding, sentence_embeddings)
most_similar_idx = similarities.argmax()  # Index of most similar sentence in training set

print(f"Most similar FAQ (from training data): {sentences[most_similar_idx]}")
print(f"Similarity score: {similarities[most_similar_idx]}")


Most similar FAQ (from training data): How to get a mortgage
Similarity score: 0.6793996095657349


In [65]:
for idx in similarities.argsort()[-5:][::-1]:
    print(f"Most similar FAQ (from training data): {sentences[idx]}")
    print(f"Similarity score: {similarities[idx]}")

Most similar FAQ (from training data): How to get a mortgage
Similarity score: 0.6793996095657349
Most similar FAQ (from training data): What’s a mortgage reserve?
Similarity score: 0.45389872789382935
Most similar FAQ (from training data): How do I pay off my mortgage?
Similarity score: 0.4509562849998474
Most similar FAQ (from training data): How much can I borrow for my mortgage?
Similarity score: 0.40523073077201843
Most similar FAQ (from training data): How do I make an overpayment to my mortgage?
Similarity score: 0.39953261613845825


In [53]:
similarities.argsort()

array([ 39, 129, 113, 160, 122,  67, 104, 128, 115, 158, 152,  46,  82,
         6,  85, 177, 146,   8, 110,  63, 135, 171,  96,   4, 174,  21,
       108,  80,  18, 154,  42, 186,  17,  19,  78,  11,  29,  55, 132,
       169, 126,   1, 179,  60, 151,  75, 109, 156, 118, 149, 178, 185,
       165, 144, 111,  33,  24,   2,  68,  72, 157,  22, 140,  45,  73,
        50,  64,  12,  59, 166,  16, 124, 175, 139,  58, 143, 150, 148,
        69,  84,  97, 170,  91, 121,  70,   0,  53,  27,  37,  48,  32,
        40, 101,  76, 145, 173, 180,  86, 147,  31,  44, 102,  93, 106,
        74, 119, 112,  79, 164,  20, 182, 161,   3, 137, 130,   5, 131,
       168,  41, 100,  34,  25,  99, 136,  38, 116,  14,  23,  47,  81,
        26, 153,  90,  15, 184, 105,  77, 159, 142, 114,  95,  10, 172,
       134, 103, 176, 181, 138,  65, 107,   7, 117, 133,  36,  57,  61,
       123,  30,  89,  54,  87, 120, 163,  28,  35, 187, 183,  51,  83,
        49,  98,  43, 167,  56,  66,  94,  52, 127,  92,   9,  7

In [38]:
print(f"Most similar FAQ (from training data): {sentences[14]}")
print(f"Similarity score: {similarities[14]}")

Most similar FAQ (from training data): How do I make an overpayment to my mortgage?
Similarity score: 0.3428468704223633


In [39]:
print(f"Most similar FAQ (from training data): {sentences[156]}")
print(f"Similarity score: {similarities[156]}")

Most similar FAQ (from training data): How can I make or cancel a CHAPS payment?
Similarity score: 0.3237667679786682


In [58]:
print(f"Most similar FAQ (from training data): {sentences[162]}")
print(f"Similarity score: {similarities[162]}")

Most similar FAQ (from training data): How to get a mortgage
Similarity score: 0.35815101861953735


In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
import gensim.downloader as api

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load pre-trained Word2Vec model
word2vec_model = api.load('word2vec-google-news-300')

# Initialize SpellChecker
spell_checker = SpellChecker()

# Initialize WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    return lemmatized_tokens

# Function to find similar words using Word2Vec
def find_similar_words(word, top_n=5):
    try:
        similar_words = word2vec_model.most_similar(word, topn=top_n)
        return [word for word, _ in similar_words]
    except KeyError:
        return []

# Function to perform spelling correction
def correct_spelling(word):
    return spell_checker.correction(word)

# Function to find synonyms using WordNet
def find_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Example usage
text = "How do I chek my accunt balnce?"
preprocessed_tokens = preprocess_text(text)
print("Preprocessed tokens:", preprocessed_tokens)

corrected_tokens = [correct_spelling(token) for token in preprocessed_tokens]
print("Spelling corrected tokens:", corrected_tokens)

similar_words = [find_similar_words(token) for token in corrected_tokens]
print("Similar words for each token:", similar_words)

synonyms = [find_synonyms(token) for token in corrected_tokens]
print("Synonyms for each token:", synonyms)
